<a href="https://colab.research.google.com/github/SahilDhull/emphasis_selection/blob/master/model/bert_as_embedding_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install config

     |████████████████████████████████| 481kB 3.3MB/s 
     |████████████████████████████████| 1.0MB 16.2MB/s 
     |████████████████████████████████| 870kB 37.4MB/s 
     |████████████████████████████████| 3.1MB 29.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=fb030b064b7f6a03ca16dfc325f3d3348aedfc547016d31ca4cc4d3ddaa092b5
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses
  Created wheel for config: filename=config-0.4.2-cp36-none-any.whl size=15135 sha256=d2e145d0cb7da6d33d7caf66e70c2996b14d0c12d188c3bc97b85bc2b7a38fee
  Stored in directory: /root/.cache/pip/wheels/51/7d/db/0e38d2ec57843d00cc39f8df3686984ccec689694f7bc78a38
Successfully built config


In [2]:
import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertConfig
from transformers import BertForMaskedLM , BertModel ,WEIGHTS_NAME, AdamW, get_linear_schedule_with_warmup
from transformers import PreTrainedModel, PreTrainedTokenizer , BertPreTrainedModel
from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
import codecs
from torch.nn.utils.rnn import pack_padded_sequence

Using TensorFlow backend.


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

'Tesla T4'

In [4]:
from google.colab import drive
drive.mount('/content/drive')

train_file = 'drive/My Drive/datasets/train.txt'
dev_file = 'drive/My Drive/datasets/dev.txt'

quotes_file = 'drive/My Drive/datasets/all_quotes.txt'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [5]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased', do_lower_case = False)

In [0]:
def read_token_map(file,word_index = 1,prob_index = 4, caseless = False):
  
  with codecs.open(file, 'r', 'utf-8') as f:
      lines = f.readlines()

  tokenized_texts = []
  token_map = []
  token_labels = []
  sent_length = []

  bert_tokens = []
  orig_to_tok_map = []
  labels = []

  bert_tokens.append("[CLS]")
  
  for line in lines:
    if not (line.isspace()):
      feats = line.strip().split()
      word = feats[word_index].lower() if caseless else feats[word_index]
      label = feats[prob_index].lower() if caseless else feats[prob_index]
      labels.append((float)(label))
      orig_to_tok_map.append(len(bert_tokens))
      
      if(word == "n't"):
        word = "'t"
        if(bert_tokens[-1] != "won"):
          bert_tokens[-1] = bert_tokens[-1] +"n"
      if(word == "wo"):
        word == "won"

      bert_tokens.extend(tokenizer.tokenize(word))
     
    elif len(orig_to_tok_map) > 0:
      bert_tokens.append("[SEP]")
      tokenized_texts.append(bert_tokens)
      token_map.append(orig_to_tok_map)
      token_labels.append(labels)
      sent_length.append(len(labels))
      bert_tokens = []
      orig_to_tok_map = []
      labels = []
      length = 0
      bert_tokens.append("[CLS]")
          
  if len(orig_to_tok_map) > 0:
    bert_tokens.append("[SEP]")
    tokenized_texts.append(bert_tokens)
    token_map.append(orig_to_tok_map)
    token_labels.append(labels)
    sent_length.append(len(labels))
  
  return tokenized_texts, token_map, token_labels, sent_length

In [7]:
t_tokenized_texts, t_token_map, t_token_label, t_sent_length = read_token_map(train_file)
print(t_tokenized_texts[100])
print(t_token_map[100])
print(t_token_label[100])
print(t_sent_length[100])

d_tokenized_texts, d_token_map, d_token_label, d_sent_length = read_token_map(dev_file)
print(d_tokenized_texts[50])
print(d_token_map[50])
print(d_token_label[50])
print(d_sent_length[50])

['[CLS]', 'Happiness', 'consists', 'in', 'realizing', 'it', 'is', 'all', 'a', 'great', 'strange', 'dream', '.', '[SEP]']
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
[0.6666666666666666, 0.1111111111111111, 0.0, 0.2222222222222222, 0.0, 0.1111111111111111, 0.1111111111111111, 0.0, 0.2222222222222222, 0.3333333333333333, 0.3333333333333333, 0.1111111111111111]
12
['[CLS]', '`', '`', 'F', '##as', '##cin', '##ating', 'social', 'media', 'tip', 'or', 'fact', 'to', 'share', '.', "'", "'", '@', 'Speaker', 'Name', '[SEP]']
[1, 3, 7, 8, 9, 10, 11, 12, 13, 14, 15, 17, 19]
[0.0, 0.5555555555555556, 0.0, 0.1111111111111111, 0.2222222222222222, 0.1111111111111111, 0.1111111111111111, 0.0, 0.2222222222222222, 0.0, 0.0, 0.2222222222222222, 0.2222222222222222]
13


In [8]:
MAX_LEN = 52

# Use the BERT tokenizer to convert the tokens to their index numbers in the BERT vocabulary
t_input_ids = [tokenizer.convert_tokens_to_ids(x) for x in t_tokenized_texts]

# Pad our input tokens
t_input_ids = pad_sequences(t_input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
t_token_map = pad_sequences(t_token_map, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
t_token_label = pad_sequences(t_token_label, maxlen=MAX_LEN, dtype="float", truncating="post", padding="post")

print(t_input_ids[100])
print(t_token_map[100])
print(t_token_label[100])

[  101 25410  2923  1107 10459  1122  1110  1155   170  1632  4020  4185
   119   102     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0]
[ 1  2  3  4  5  6  7  8  9 10 11 12  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0]
[0.66666667 0.11111111 0.         0.22222222 0.         0.11111111
 0.11111111 0.         0.22222222 0.33333333 0.33333333 0.11111111
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         

In [9]:
d_input_ids = [tokenizer.convert_tokens_to_ids(x) for x in d_tokenized_texts]

# Pad our input tokens
d_input_ids = pad_sequences(d_input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
d_token_map = pad_sequences(d_token_map, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
d_token_label = pad_sequences(d_token_label, maxlen=MAX_LEN, dtype="float", truncating="post", padding="post")

print(d_input_ids[50])
print(d_token_map[50])
print(d_token_label[50])

[  101   169   169   143  2225 16430  3798  1934  2394  5580  1137  1864
  1106  2934   119   112   112   137  9911 10208   102     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0]
[ 1  3  7  8  9 10 11 12 13 14 15 17 19  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0]
[0.         0.55555556 0.         0.11111111 0.22222222 0.11111111
 0.11111111 0.         0.22222222 0.         0.         0.22222222
 0.22222222 0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         

In [10]:
t_attention_masks = []
# Create a mask of 1s for each token followed by 0s for padding
for seq in t_input_ids:
  seq_mask = [float(i>0) for i in seq]
  t_attention_masks.append(seq_mask)
print(t_attention_masks[100])

d_attention_masks = []
# Create a mask of 1s for each token followed by 0s for padding
for seq in d_input_ids:
  seq_mask = [float(i>0) for i in seq]
  d_attention_masks.append(seq_mask)
print(d_attention_masks[50])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [0]:
t_input_ids = torch.tensor(t_input_ids)
t_token_map = torch.tensor(t_token_map )
t_token_label = torch.tensor(t_token_label)
t_attention_masks = torch.tensor(t_attention_masks)
t_sent_length = torch.tensor(t_sent_length)

d_input_ids = torch.tensor(d_input_ids)
d_token_map = torch.tensor(d_token_map )
d_token_label = torch.tensor(d_token_label)
d_attention_masks = torch.tensor(d_attention_masks)
d_sent_length = torch.tensor(d_sent_length)

# Select a batch size for training. 
batch_size = 32
# print(t_token_labels)
# Create an iterator of our data with torch DataLoader 
train_data = TensorDataset(t_input_ids, t_token_map, t_token_label, t_attention_masks, t_sent_length)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
validation_data = TensorDataset(d_input_ids, d_token_map, d_token_label, d_attention_masks, d_sent_length)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [0]:
class bert_model(nn.Module):
  def __init__(self, final_size, drop_prob, data_parallel=True):
    super(bert_model, self).__init__()

    config = BertConfig.from_pretrained('bert-base-cased', output_hidden_states=True)
    bert = BertForMaskedLM.from_pretrained('bert-base-cased', config = config)
    
    if data_parallel:
        self.bert = nn.DataParallel(bert)
    else:
        self.bert = bert
    bert_dim = 768
    hidden_dim = 20

    self.fc1 = nn.Linear(bert_dim, hidden_dim)
    self.fc2 = nn.Linear(hidden_dim, final_size)
    self.dropout1 = nn.Dropout(p=drop_prob)
    self.dropout2 = nn.Dropout(p=drop_prob)
           
  def forward(self, bert_ids, bert_mask, labels = None, bert_token_starts = None,lm_lengths = None):
    
    batch_size = bert_ids.size()[0]
    pad_size = bert_ids.size()[1]
    # print("batch size",batch_size,"\t\tpad_size",pad_size)

    if(bert_token_starts == None):
      output = self.bert(bert_ids, attention_mask = bert_mask, masked_lm_labels=labels)
      return output
    
    output = self.bert(bert_ids, attention_mask = bert_mask)
    # print(len(output))
    # print(len(output[1]))
    # print(output[1][0].size())
    bert_last_layer = output[1][0]
    
    pred_logits = torch.relu(self.fc1(self.dropout1(bert_last_layer)))
    pred_logits = torch.sigmoid(self.fc2(self.dropout2(pred_logits)))
    pred_logits = torch.squeeze(pred_logits,2)
    # print(pred_logits.size())
    # print(labels.size())
    # print(pred_logits[1])
    # print(labels[1])
    # print(bert_token_starts[1])
    # print("\n")

    pred_labels = labels.clone()
    # print(pred_labels[1])
    # print("\n")
    
    for b in range(batch_size):
      for w in range(pad_size):
        if(bert_token_starts[b][w]!=0):
          if(bert_token_starts[b][w]>=pad_size):
            print(bert_token_starts[b])
          else:
            pred_labels[b][w] = pred_logits[b][bert_token_starts[b][w]]

    # print(pred_labels[1])
    # print(labels[1])
    # print("\n")

    lm_lengths, lm_sort_ind = lm_lengths.sort(dim=0, descending=True)
    scores = labels[lm_sort_ind]
    targets = pred_labels[lm_sort_ind]
    scores = pack_padded_sequence(scores, lm_lengths, batch_first=True).data
    targets = pack_padded_sequence(targets, lm_lengths, batch_first=True).data
    
    # mask = pred_labels!=0
    # total = mask[mask].size()[0]

    # loss_fn = nn.BCELoss(reduction='sum').to(device) 
    loss_fn = nn.BCELoss().to(device) 
    loss = loss_fn(targets,scores)
    # print(loss)

    # loss /= total 
    # print(loss) 
    return loss, pred_labels

In [0]:
model = bert_model(1,0.3,True).to(device)

In [0]:
optimizer = AdamW(model.parameters(), lr=2e-4, eps = 1e-8)

epochs = 20
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [0]:
def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3

def fix_padding(scores_numpy, label_probs,  mask_numpy):
    #if len(scores_numpy) != len(mask_numpy):
    #    print("Error: len(scores_numpy) != len(mask_numpy)")
    #assert len(scores_numpy) == len(mask_numpy)
    #if len(label_probs) != len(mask_numpy):
    #    print("len(label_probs) != len(mask_numpy)")
    #assert len(label_probs) == len(mask_numpy)

    all_scores_no_padd = []
    all_labels_no_pad = []
    for i in range(len(mask_numpy)):
        all_scores_no_padd.append(scores_numpy[i][:int(mask_numpy[i])])
        all_labels_no_pad.append(label_probs[i][:int(mask_numpy[i])])

    assert len(all_scores_no_padd) == len(all_labels_no_pad)
    return all_scores_no_padd, all_labels_no_pad

def match_M(batch_scores_no_padd, batch_labels_no_pad):

    top_m = [1, 2, 3, 4]
    batch_num_m=[]
    batch_score_m=[]
    for m in top_m:
        intersects_lst = []
        # exact_lst = []
        score_lst = []
        ############################################### computing scores:
        for s in batch_scores_no_padd:
            if len(s) <=m:
                continue
            h = m
            # if len(s) > h:
            #     while (s[np.argsort(s)[-h]] == s[np.argsort(s)[-(h + 1)]] and h < (len(s) - 1)):
            #         h += 1

            # s = np.asarray(s.cpu())
            s = np.asarray(s)
            #ind_score = np.argsort(s)[-h:]
            ind_score = sorted(range(len(s)), key = lambda sub: s[sub])[-h:]
            score_lst.append(ind_score)

        ############################################### computing labels:
        label_lst = []
        for l in batch_labels_no_pad:
            if len(l) <=m:
                continue
            # if it contains several top values with the same amount
            h = m
            # l = l.cpu()
            if len(l) > h:
                while (l[np.argsort(l)[-h]] == l[np.argsort(l)[-(h + 1)]] and h < (len(l) - 1)):
                    h += 1
            l = np.asarray(l)
            ind_label = np.argsort(l)[-h:]
            label_lst.append(ind_label)

        ############################################### :

        for i in range(len(score_lst)):
            intersect = intersection(score_lst[i], label_lst[i])
            intersects_lst.append((len(intersect))/(min(m, len(score_lst[i]))))
            # sorted_score_lst = sorted(score_lst[i])
            # sorted_label_lst =  sorted(label_lst[i])
            # if sorted_score_lst==sorted_label_lst:
            #     exact_lst.append(1)
            # else:
            #     exact_lst.append(0)
        batch_num_m.append(len(score_lst))
        batch_score_m.append(sum(intersects_lst))
    return batch_num_m, batch_score_m

In [0]:
def validation(model, validation_dataloader):
  print("")
  print("Running Validation...")

  model.eval()
  eval_loss, eval_accuracy = 0, 0
  nb_eval_steps, nb_eval_examples = 0, 0

  num_m = [0, 0, 0, 0]
  score_m = [0, 0, 0, 0]

  # Evaluate data for one epoch
  for batch in validation_dataloader:
      
      # Add batch to GPU
      batch = tuple(t.to(device) for t in batch)
      
      # Unpack the inputs from our dataloader
      v_input_ids = batch[0].to(device)
      v_input_mask = batch[3].to(device)
      v_token_starts = batch[1].to(device)
      v_labels = batch[2].to(device)
      v_sent_length = batch[4]
            
      # Telling the model not to compute or store gradients, saving memory and
      # speeding up validation
      with torch.no_grad():        
          output = model(v_input_ids, v_input_mask, v_labels, v_token_starts, v_sent_length)
      
      pred_labels = output[1]

      pred_labels = pred_labels.detach().cpu().numpy()
      v_labels = v_labels.to('cpu').numpy()
      # print(pred_labels[0])
      # print(v_labels[0])
      
      pred_labels, v_labels = fix_padding(pred_labels, v_labels, v_sent_length)
      # print(pred_labels[0])
      # print(v_labels[0])

      batch_num_m, batch_score_m = match_M(pred_labels, v_labels)
      num_m = [sum(i) for i in zip(num_m, batch_num_m)]
      score_m = [sum(i) for i in zip(score_m, batch_score_m)]
  
  m_score = [i/j for i,j in zip(score_m, num_m)]
  print("Validation Accuracy: ")
  print(m_score)
  v_score = np.mean(m_score)
  print(v_score)

In [34]:

# import random

# # Set the seed value all over the place to make this reproducible.
# seed_val = 42

# random.seed(seed_val)
# np.random.seed(seed_val)
# torch.manual_seed(seed_val)
# torch.cuda.manual_seed_all(seed_val)

# Store the average loss after each epoch so we can plot them.
loss_values = []

# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Reset the total loss for this epoch.
    total_loss = 0
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):
            
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[3].to(device)
        b_token_starts = batch[1].to(device)
        b_labels = batch[2].to(device)
        b_sent_length = batch[4]

        model.zero_grad()   
        model.train()     

        output = model(b_input_ids, b_input_mask, b_labels, b_token_starts,b_sent_length)
        loss = output[0]

        total_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

        if step % 20 == 0:
          validation(model, validation_dataloader)

    # Calculate the average loss over the training data.
    # print("total loss",total_loss)
    avg_train_loss = total_loss / len(train_dataloader)            
    
    # Store the loss value for plotting the learning curve.
    loss_values.append(avg_train_loss)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))


print("")
print("Training complete!")


======== Epoch 1 / 20 ========
Training...

Running Validation...
Validation Accuracy: 
[0.16071428571428573, 0.29708222811671087, 0.3873106060606061, 0.4621212121212121]
0.3268070830032037

Running Validation...
Validation Accuracy: 
[0.35459183673469385, 0.5689655172413793, 0.6941287878787878, 0.7393939393939394]
0.5892700203122001

Running Validation...
Validation Accuracy: 
[0.4107142857142857, 0.6021220159151194, 0.7102272727272727, 0.7583333333333333]
0.6203492269225028

Running Validation...
Validation Accuracy: 
[0.43112244897959184, 0.6273209549071618, 0.7339015151515152, 0.7765151515151515]
0.6422150176383551

Running Validation...
Validation Accuracy: 
[0.46938775510204084, 0.6485411140583555, 0.7433712121212122, 0.781060606060606]
0.6605901718355536

  Average training loss: 0.56

======== Epoch 2 / 20 ========
Training...

Running Validation...
Validation Accuracy: 
[0.461734693877551, 0.6551724137931034, 0.7395833333333333, 0.7780303030303031]
0.6586301860085727

Running